In [1]:
import subprocess
from pathlib import Path

def my_system(cmd):
    return subprocess.run(cmd, capture_output=True).stdout.decode("utf-8").split("\n")

def find(pattern, including=None):
    a = my_system(["find", ".", "-name", pattern])
    a = [l for l in a if len(l) != 0 and Path(l).is_file()]
    if including == None:
        return a
    else: 
        res = []
        for l in a:
            if subprocess.run(["grep", "-q", including, l]).returncode == 0:
                res.append(l)
        return res

def rename(fromf, tof):
#    subprocess.run(["git", "mv", fromf, tof])
    subprocess.run(["mv", fromf, tof])

def replace(fn, froms, tos):
    froms = froms.replace("/", "\/")
    tos = tos.replace("/", "\/")
    cmd = f"s/{froms}/{tos}/g"
    subprocess.run(["sed", "-i", "", cmd, fn])

def nkf(fn):
    subprocess.run(["nkf", "--in-place", fn])

In [4]:
for fn in find("*.html"):
    replace(fn, "http://xsig.hpcc.jp/", "/")

#    replace(fn, "https://xsig.ipsj.or.jp/", "/")
#    replace(fn, "http://xsig.ipsj.or.jp/", "/")

In [16]:
find("*.html.en")



['./2013/cfp.html.en']

In [5]:
import os
os.chdir("docs")

In [8]:
for fn in find("*.html.ja"):
    nkf(fn)
    replace(fn, "EUC-JP", "UTF8")
    replace(fn, ".html.ja", ".ja.html")
    replace(fn, ".html.en", ".en.html")
    rename(fn, fn.replace(".html.ja", ".ja.html"))
        

In [18]:
for fn in find("*.html.en"):
    nkf(fn)
    replace(fn, "EUC-JP", "UTF8")
    replace(fn, ".html.ja", ".ja.html")
    replace(fn, ".html.en", ".en.html")
    rename(fn, fn.replace(".html.en", ".en.html"))

In [5]:
for fn in find("*.html"):
    nkf(fn)
    replace(fn, "EUC-JP", "UTF8")
    replace(fn, ".html.ja", ".ja.html")
    replace(fn, ".html.en", ".en.html")    

In [13]:
from pathlib import Path
import re
pat = re.compile(r"""<\?php include\("(.*)"\);\?>""")


def process_one_line(parent_dir, line):
    m = pat.search(line)
    if m:
        path = parent_dir / m[1]
        a = phpInclude(path, p_dir=parent_dir)
        return ["<!-- included from " + m[1] + " Start -->\n"] + a + ["<!-- included from " + m[1] + " END -->\n"] 
    else:
        return [line]


def phpInclude(fn, p_dir=None):
    '''returns array of lines with \n'''
    if p_dir == None:
        p_dir = Path(fn).parent
    try:
        with open(fn) as f:
            res = []
            for line in f.readlines():
                res += process_one_line(p_dir, line)
        return res
    except FileNotFoundError:
        print(f"cannot find {fn}, ignore")
        return []

def dump(filename, array):
    with open(filename, "w") as f:
        for line in array:
            f.write(line)

def include_and_replace(fn):
    res = phpInclude(fn)
    rep_fn = fn+".replaced"
    dump(rep_fn, res)
    subprocess.run(["mv", rep_fn, fn])

        

In [19]:
filenames = \
    find("*.html",    including="php") + \
    find("*.html.ja", including="php") + \
    find("*.html.en", including="php")

for fn in filenames:
    print(fn)
    include_and_replace(fn)

./2013/cfp.en.html
cannot find 2013/include/theme-en.html, ignore
./2009/cell/outline.html
./2004/inner/index.html


In [77]:
dump("test.html", res)

In [14]:
include_and_replace("2013/test.html")


cannot find 2013/include/theme-en.html, ignore


In [10]:
for fn in find("*.html", "\"/history/"):
    replace(fn, "\"/history/", "\"https://archive.xsig.ipsj.or.jp/history/")

In [11]:
find("*.html", "\"/history/")

['./2022/about/history.html',
 './2022/en/about/history.html',
 './2021/about/history.html',
 './2021/en/about/history.html']

In [36]:
for fn in find("*.html", "phase"):
    replace(fn, "http://phase.hpcc.jp/phase/sacsis/SACSIS2003/archives", "http://sacsis.hpcc.jp/2003/")

In [25]:
replace("2013/test.html", "http://phase.hpcc.jp/phase/sacsis/SACSIS2003/archives/", "http://sacsis.hpcc.jp/2003/")

In [37]:
import os
os.chdir("../JSPP")



In [38]:
for fn in find("*.html"):
    replace(fn, "iso-2022-jp", "UTF8")